In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda

llm = ChatOpenAI(
    temperature=0.1,
)

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

loader = UnstructuredFileLoader("./files/chapter_one.docx")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embeddings, cache_dir
)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

retriever = vectorstore.as_retriever()

map_doc_prompt = ChatPromptTemplate.from_messages([
    (
        "system", 
        """
            Use the following portion of a long document to see if any of the text is relevant to answer the question.
            Return any relevant text verbatim.
            ------
            {context}
        """
     ),
    ("human", "{question}")
])

map_doc_chain = map_doc_prompt | llm

def map_docs(inputs):
    documents = inputs["documents"]
    question = inputs["question"]
    return "\n\n".join(map_doc_chain.invoke({"context": document.page_content, "question": question}).content for document in documents)

map_chain = {"documents": retriever, "question": RunnablePassthrough()} | RunnableLambda(map_docs)

final_prompt = ChatPromptTemplate.from_messages([
    (
        "system", 
        """
            Given the folling extracted parts of a long document and a question, create a final answer.
            If you don't know the answer just say you don't know. Don't try to make up answer.
            ------
            {context}
        """
    ),
    ("human", "{question}")
])

chain = {"question": RunnablePassthrough(), "context": map_chain} | final_prompt | llm


AIMessage(content='Victory Mansions is a building with glass doors that Winston Smith enters. The hallway smells of boiled cabbage and old rag mats. It has seven flights of stairs, and the lift is usually not working. There is a large colored poster of a man\'s face with the caption "BIG BROTHER IS WATCHING YOU" on one end of the hallway. Inside the flat, there is a telescreen that cannot be completely shut off, and a fruity voice reading out figures related to pig-iron production.')